In [ ]:
import sys
import os
import csv
import io
from pprint import pprint
import json

sys.path.insert(0, os.path.abspath('../src'))
import utilities as utils

In [ ]:
# Read in config file
config_file = '/Users/mslivins/Projects/redcap-image-processing/src/validation.env'
config = utils.read_env(config_file)

In [ ]:
# Request single record
test_record = '94'
data = {
    'token': config['API_TOKEN'],
    'content': 'record',
    'action': 'export',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'records[0]': test_record,
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'csv'
}
r = utils.post_request(config['API_URL'], data)
print('HTTP Status: ' + str(r.status_code))
assert str(r.status_code) == '200', "Need 200 successful response"

# Convert CSV request to dict records
records = utils.records_to_dict(r.text)

# Request single image upload
test_img_upload = 'bl_c_img_view_1'
data = {
    'token': config['API_TOKEN'],
    'content': 'file',
    'action': 'export',
    'record': test_record,
    'field': test_img_upload,
    'event': '',
    'returnFormat': 'json'
}
r = utils.post_request(config['API_URL'], data)
print('HTTP Status: ' + str(r.status_code))
assert str(r.status_code) == '200', "Need 200 successful response"

# Convert image file bytes to Image object 
image = utils.bytes_to_image(r.content)
# image.show()

# Add image to record
# NOTE: testing only for single record a for a single image upload
img_name = records[test_record][test_img_upload]
records[test_record][test_img_upload] = {img_name: image}

In [ ]:
# TODO: need to recreate annotations based on original image size


# For each circled annotation, check if single annotation is within circled area
# NOTE: We assume there is only one annotation area per set of individual 
# annotated scars.
# TODO: this will need to change to handle multiple annotation areas
for record_id, record in records.items():
    # Get all annotation keys
    annot_keys = [key for key in list(record.keys()) if 'annotate' in key]
    # Convert string JSON into dictionary
    annot_dict = {
        annot_key: {} if record[annot_key] == '' else json.loads(record[annot_key])
        for annot_key in annot_keys
    }
    
    
    # Get single area annotation key
    # area_key = [annot_key for annot_key in annot_keys if 'annotate_all' in annot_key][0]
    # Remove area annotation key from the other annotation keys
    # annot_keys = list(set(annot_keys) - set([area_key]))
    # Convert string JSON into dictionary
    # print(annot_keys)

# print(
#     json.loads(records['94']['bl_c_img_flf_annotate_all'])
# )


In [ ]:
# help(r)

In [ ]:
# For each circled annotation, check if single annotation is within circled area